<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import plotly.express as px
import matplotlib.pyplot as plt
from unicodedata import normalize
import warnings
warnings.filterwarnings('ignore')

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = f'''SELECT
    COUNT(id)
FROM vacancies
            '''

In [5]:
df_3_1 = pd.read_sql_query(query_3_1, connection)
print('Всего в базе находится',df_3_1.iloc[0][0], 'вакансий.')

Всего в базе находится 49197 вакансий.


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query_3_2 = f'''SELECT
    COUNT(id)
FROM employers
            '''

In [7]:
df_3_2 = pd.read_sql_query(query_3_2, connection)
print('База содержит', df_3_2.iloc[0][0], 'работодателей.')

База содержит 23501 работодателей.


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query_3_3 = f'''SELECT
    COUNT(id)
FROM areas
            '''

In [9]:
df_3_3 = pd.read_sql_query(query_3_3, connection)
print('В базе данных представлено', df_3_3.iloc[0][0], 'региона.')

В базе данных представлено 1362 региона.


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query_3_4 = f'''SELECT
    COUNT(id)
FROM industries
            '''

In [11]:
df_3_4 = pd.read_sql_query(query_3_4, connection)
print('База содержит вакансии в', df_3_4.iloc[0][0], 'сферах деятельности.')

База содержит вакансии в 294 сферах деятельности.


***

<h4>Выводы по первичному анализу.</h4>

Первичный анализ данных показал, что в нашей базе данных содержатся сведения о 49197 вакансиях. 
Также я свела данные из таблиц о вакансиях и работодателях, и увидела, что представлен 23 501 работодатель, а значит, многим из них требуется не один сотрудник, что в принципе логично. Интересно, есть ли работодатели, не разместившие ни одной вакансии? Но я рассмотрю это дальше при более глубоком исследовании.
В базе данных представлено 1362 региона.  Наверняка, вакансии есть не во всех из них, обратим на это внимание при дальнейшем исследовании.
Также мы узнали, что в базе данных содержится информация о вакансиях в 294 сферах деятельности. 
Думаю, с данными будет интересно работать. Посмотрим, что полезного и интересного можно из них извлечь.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_4_1 = f'''SELECT
    a.name area,
    COUNT(v.id) cnt
FROM vacancies v
JOIN areas a ON v.area_id = a.id
GROUP BY area
ORDER BY cnt DESC
            '''

In [13]:
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_4_2 = f'''SELECT
    COUNT(id)
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
        '''

In [15]:
df_4_2 = pd.read_sql_query(query_4_2, connection)
print(f'Хотя бы одно из двух полей с зарплатой заполнено у {df_4_2.iloc[0][0]} вакансий.')

Хотя бы одно из двух полей с зарплатой заполнено у 24073 вакансий.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query_4_3 = f'''SELECT
    ROUND(AVG(salary_from)) salary_from,
    ROUND(AVG(salary_to)) salary_to
FROM vacancies
            '''

In [17]:
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,salary_from,salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query_4_4 = f'''SELECT
    schedule,
    employment,
    COUNT(id) cnt
FROM vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC
            '''

In [19]:
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_4_5 = f'''SELECT
    experience,
    COUNT(id)
FROM vacancies
GROUP BY(experience)
ORDER BY COUNT(id)
            '''

In [21]:
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

<h4>Выводы по анализу вакансий.</h4>

1. Сразу же мы видим, что в данных представлены вакансии в 769 регионах, в то время как общее количество регионов в наших данных - 1362. Почти в половине городов нет вакансий. Больше всего рабочих мест в крупных городах, лидером является Москва, здесь 5333 вакансии.

2. Далее мы обнаружили, что у большинства вакансий не заполнено ни одно поле с зарплатой. Лишь у 24073 вакансий из 49197 заполнено хотя бы одно из полей с зарплатой. Средние значения для нижней и верхней границы зарплатной вилки составляют 71065 рублей и 110 537 рублей соответственно.

3. Мы также узнали, какие сочетания типа рабочего графика и типа трудоустройства наиболее популярны у работодателей. Преобладающее большинство (35367) вакансий содержат указания на полный день и полную занятость. На удаленку при полной занятости согласны работодатели, разместившие 7802 вакансии, гибкий график при полной занятости предлагается в 1593 вакансиях, удаленная работа с частичной занятостью - в 1312 предложениях о работе.
Совсем непопулярным оказалось сочетание сменного графика с проектной работой - 1 вакансия.
4. Относительно опыта работы: наибольшее количество предложений требуют опыт работы 1 года до 3 лет (26152 вакансии из 49197, это больше половины), за ними следуют "от 3 до 6 лет" - 14511 вакансий, совсем без опыта можно выбрать работу среди 7197 вариантов, а вот очень важен опыт и его требуется более 6 лет - в 1337 вакансиях.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query_5_1 = f'''SELECT
    e.name,
    COUNT(v.id) cnt
FROM employers e
JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.name
ORDER BY cnt DESC
--LIMIT 5
        '''

In [23]:
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14761,им. Тьюринга,1
14762,LLC Novalab Tech,1
14763,Компания Мир упаковки,1
14764,Лифинцев Олег Иванович,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
query_5_2 = f'''SELECT
    DISTINCT(a.name) area_name,
    COUNT(DISTINCT e.id) emp_cnt,
    COUNT(DISTINCT v.id) vac_cnt
FROM areas a
LEFT JOIN employers e ON e.area = a.id
LEFT JOIN vacancies v ON a.id = v.area_id
WHERE v.id IS NULL
GROUP BY area_name
ORDER BY 2 DESC
            '''

In [25]:
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

,area_name,emp_cnt,vac_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Юрюзань,0,0
589,Ямбург (Ямало-Ненецкий АО),0,0
590,Янино-1,0,0
591,Ясный (Оренбургская область),0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query_5_3 = f'''SELECT
    DISTINCT(e.name) emp_name,
    COUNT(DISTINCT v.area_id)
FROM employers e
LEFT JOIN vacancies v ON v.employer_id = e.id
GROUP BY emp_name
ORDER BY 2 DESC
            '''

In [27]:
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,emp_name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,Ярпож Казань,0
23171,ЯРСНИП,0
23172,Ясли-сад № 28 г.Минска,0
23173,Яшин&Партнёры,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
query_5_4 = f'''SELECT
    COUNT(DISTINCT e.id)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
LEFT JOIN industries i ON i.id = ei.industry_id
WHERE i.id IS NULL
            '''

In [29]:
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
query_5_5 = f'''SELECT
    e.name emp_name,
    COUNT(i.id) ind_cnt
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
LEFT JOIN industries i ON i.id = ei.industry_id
GROUP BY emp_name
HAVING COUNT(i.id) = 4
ORDER BY 1
            '''

In [31]:
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,emp_name,ind_cnt
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.

In [32]:
query_5_6 = f'''SELECT
    DISTINCT i.name ind_name,
    COUNT(e.id)
FROM employers e
JOIN employers_industries ei ON e.id = ei.employer_id
JOIN industries i ON i.id = ei.industry_id
WHERE i.name = 'Разработка программного обеспечения'
GROUP BY ind_name
            '''

In [33]:
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

,ind_name,count
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# получим кортеж городов-миллионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%9A%D1%80%D1%83%D0%BF%D0%BD%D0%B5%D0%B9%D1%88%D0%B8%D0%B5_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D1%81%D1%83%D0%BC%D0%BC%D0%B0%D1%80%D0%BD%D0%BE'
tabs = pd.read_html(url, match='43,8%')
display(tabs[0])
cities_df = tabs[0]
cities = list(cities_df['Город'])
cities[-3] = 'Воронеж'
cities = tuple(cities)
print(cities)

,№,Город,"Население, тыс. чел. (итоги переписи 2021 года)[5]",Прирост относительно переписи-2010,"Население, тыс. чел.(итоги переписи 2010 года)[6]"
0,1,Москва,13 010,"13,1%",11 504
1,2,Санкт-Петербург,5602,"14,8%",4880
2,3,Новосибирск,1634,"10,9%",1474
3,4,Екатеринбург,1544,"14,4%",1350
4,5,Казань,1309,"14,4%",1144
5,6,Нижний Новгород,1226,NaN,1251
6,7,Челябинск,1190,"5,3%",1130
7,8,Красноярск,1188,"21,6%",977
8,9,Самара,1173,"0,3%",1169
9,10,Уфа,1145,"7,8%",1062


('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [35]:
query_5_7 = f'''SELECT
    a.name "Город",
    COUNT(v.id) "Количество вакансий"
FROM areas a
JOIN vacancies v ON a.id = v.area_id
JOIN employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс'
    AND (a.name IN {cities})
GROUP BY "Город"

UNION ALL

SELECT    
    'total',
    COUNT(v.id)
FROM areas a
JOIN vacancies v ON a.id = v.area_id
JOIN employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс'
    AND (a.name IN {cities})
    
ORDER BY "Количество вакансий"
    '''

In [36]:
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,Город,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

<h4>Выводы по анализу работодателей.</h4>

1. Посмотрим, какие работодатели разместили больше всего вакансий.
- На первом месте находится компания Яндекс, которой было размещено 1933 вакансии, что довольно много даже на фоне общего количества вакансий(49197).
- В пятерку лидеров по числу вакансий вошли также Ростелеком(491), Тинькофф(444), СБЕР(428), Газпром нефть(331).

2. При рассмотрении распределения вакансий каждого работодателя выяснилось, что Яндекс и здесь лидирует, вакансии этой компании размещены для 181 региона. 
На втором месте Ростелеком, охвативший 152 города, на третьем Спецремонт(116 городов).

3. При более детальном рассмотрении работодателей выяснилось, что у 8419 из 23501 не указана сфера деятельности. Зато есть компании, у которых указано по несколько сфер деятельности. Так вот, 1136 работодателей указали по 4 сферы деятельности для себя.

4. Мы выяснили, что у 3553 компаний в качестве сферы деятельности указана Разработка программного обеспечения. Мне хотелось бы рассмотреть эту сферу деятельности в разрезе указанных зарплат, я это сделаю в итоговом выводе.

5. Далее мы взяли с википедии список городов миллионников и посмотрели, какое количество вакансий компании Яндекс представлено в городах-миллионниках. Всего в таких городах размещено 485 вакансий (всего у Яндекса 1933 вакансии) компании, больше всего - в Москве(54 предложения). Интересно, почему у Яндекса такой разброс по городам? Я обязательно исследую этот момент и укажу на него в итоговом выводе.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
query_6_1 = f'''SELECT COUNT(id)
FROM vacancies
WHERE LOWER(name) LIKE '%data%'
    OR LOWER(name) LIKE '%данн%'
        '''

In [38]:
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
query_6_2 = f'''SELECT 
    COUNT(id) cnt
FROM vacancies
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR name ILIKE lower('%ML%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
    AND (name ILIKE '%junior%'
    OR experience ILIKE lower('%Нет опыта%')
    OR employment ILIKE lower('%Стажировка%'))
    AND name NOT ILIKE '%html%'
    '''

In [40]:
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
query_6_3 = f'''SELECT 
    COUNT(id) cnt
FROM vacancies
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name LIKE '%ML%' AND name NOT ILIKE '%html%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
    AND (key_skills LIKE '%SQL%'
    OR key_skills LIKE '%postgres%')
    '''

In [42]:
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

,cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
query_6_4 = f'''SELECT 
    COUNT(id) cnt
FROM vacancies
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name LIKE '%ML%' AND name NOT ILIKE '%html%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
    AND key_skills ILIKE '%Python%'
    '''

In [44]:
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

,cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
query_6_5 = f'''SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1),2)
FROM vacancies
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name LIKE '%ML%' AND name NOT ILIKE '%html%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
'''

In [46]:
df_6_5 = pd.read_sql_query(query_6_5, connection)
df_6_5

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
query_6_6 = f'''SELECT
    DISTINCT experience,
    ROUND(AVG(COALESCE(((salary_from + salary_to) / 2), salary_from, salary_to)))
FROM 
    vacancies
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name LIKE '%ML%' AND name NOT ILIKE '%html%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
    AND (salary_from IS NOT NULL
    OR salary_to IS NOT NULL)
GROUP BY experience
'''

In [48]:
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

<h4>Выводы по предметному анализу</h4>

1. В нашей базе данных нашлась 1771 вакансия, имеющая отношение к данным. Для начинающего дата-сайентиста в базе нашлась 51 вакансия, и, на мой взгляд, это не так плохо, могло быть и меньше.

2. Если рассматривать не джуниорские позиции, а обычные вакансии DS, то в 201 из них в качестве ключевых навыков указаны SQL или postgres. Python требуется чаще - в 351 аналогичной вакансии. В среднем, если округлить, в вакансиях для DS указано 6 навыков.

3. Какие зарплаты ждут дата-сайентистов с разным уровнем работы?
- Нет опыта - 74643.0 рублей.
- От 1 года до 3 лет - 139675.0 рублей.
- От 3 до 6 лет  - 243115 рублей.
Куда делись DS-вакансии с опытом работы от 6 лет? Я проведу дополнительное исследование и укажу на это в итоговом выводе.

# Общий вывод по проекту

In [49]:
# посмотрим, как выглядят основные данные из таблицы с вакансиями
query_7_1 = f'''SELECT
    v.id "ID вакансии",
    v.schedule "Тип рабочего графика",
    v.experience "Требования к опыту",
    v.salary_from "Нижняя граница зарплаты",
    v.salary_to "Верхняя граница зарплаты"
       
FROM vacancies v
        '''

df_7_1 = pd.read_sql_query(query_7_1, connection)
display(df_7_1)


,ID вакансии,Тип рабочего графика,Требования к опыту,Нижняя граница зарплаты,Верхняя граница зарплаты
0,55312386,Полный день,Нет опыта,64000.0,NaN
1,55843014,Полный день,От 3 до 6 лет,NaN,NaN
2,54525964,Удаленная работа,От 3 до 6 лет,NaN,NaN
3,54525965,Удаленная работа,От 3 до 6 лет,NaN,NaN
4,55354053,Удаленная работа,Нет опыта,15000.0,NaN
...,...,...,...,...,...
49192,55589514,Полный день,От 3 до 6 лет,NaN,NaN
49193,55029668,Полный день,От 3 до 6 лет,NaN,NaN
49194,66172263,Полный день,От 1 года до 3 лет,60000.0,NaN
49195,66180086,Полный день,От 1 года до 3 лет,146456.0,244094.0


In [50]:
# посмотрим, какие типы данных содержатся в таблице и сколько непустых значений в столбцах
df_7_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49197 entries, 0 to 49196
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID вакансии               49197 non-null  int64  
 1   Тип рабочего графика      49197 non-null  object 
 2   Требования к опыту        49197 non-null  object 
 3   Нижняя граница зарплаты   21485 non-null  float64
 4   Верхняя граница зарплаты  12690 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.9+ MB


In [51]:
# соберем статистику по числовым столбцам
df_7_1.describe()

,ID вакансии,Нижняя граница зарплаты,Верхняя граница зарплаты
count,4.919700e+04,21485.000000,12690.000000
mean,5.535701e+07,71064.657901,110536.741923
std,4.215213e+06,56646.897132,89104.795997
min,6.555939e+06,30.000000,60.000000
25%,5.465343e+07,35000.000000,50000.000000
50%,5.512157e+07,50000.000000,80000.000000
75%,5.544978e+07,90000.000000,150000.000000
max,6.619644e+07,1000000.000000,1000000.000000


In [52]:
# построим диаграмму распределения нижней границы зп в зависимости от опыта

fig = px.box(
    data_frame=df_7_1,
    x = 'Нижняя граница зарплаты',
    y = 'Требования к опыту',
    color = 'Требования к опыту',
    height= 500,
    width= 900,
    title= 'Распределение нижней границы зп в зависимости от опыта'
)
fig.show()

In [53]:
# вакансии с сочетанием сменного графика или вахтового метода с проектной работой

query_7_2 = f'''SELECT
    v.name "Название вакансии",
    v.key_skills "Ключевые навыки",
    v.schedule "Тип рабочего графика",
    v.experience "Требования к опыту",
    v.employment "Тип трудоустройства",
    v.salary_from "Нижняя граница зарплаты",
    v.salary_to "Верхняя граница зарплаты",
    a.name "Название города",
    e.name "Название работодателя"
    
FROM vacancies v
JOIN areas a ON a.id = v.area_id
JOIN employers e ON e.id = v.employer_id

WHERE (v.schedule = 'Сменный график' OR v.schedule = 'Вахтовый метод')
AND v.employment = 'Проектная работа'
        '''
        
df_7_2 = pd.read_sql_query(query_7_2, connection)
df_7_2

,Название вакансии,Ключевые навыки,Тип рабочего графика,Требования к опыту,Тип трудоустройства,Нижняя граница зарплаты,Верхняя граница зарплаты,Название города,Название работодателя
0,DevOPS Engineer,Английский язык\tУправление отношениями с клие...,Вахтовый метод,От 3 до 6 лет,Проектная работа,NaN,NaN,Атырау,Norsec Delta Projects
1,Продюсер,Организация конференций\tКонтроль качества\tУп...,Сменный график,Нет опыта,Проектная работа,50000.0,70000.0,Санкт-Петербург,JUG Ru Group
2,Инженер по релейной защите и автоматике,AutoCAD\tПользователь ПК\tЧтение электросхем\t...,Вахтовый метод,От 3 до 6 лет,Проектная работа,150000.0,NaN,Челябинск,ЭнергоРемонтСервис


In [54]:
# вакансии с зп >=500000

query_7_3 = f'''SELECT
    v.name "Название вакансии",
    v.key_skills "Ключевые навыки",
    v.schedule "Тип рабочего графика",
    v.experience "Требования к опыту",
    v.employment "Тип трудоустройства",
    v.salary_from "Нижняя граница зарплаты",
    v.salary_to "Верхняя граница зарплаты",
    a.name "Название города",
    e.name "Название работодателя",
    i.name "Название сферы деятельности"
    
FROM vacancies v
JOIN areas a ON a.id = v.area_id
JOIN employers e ON e.id = v.employer_id
JOIN employers_industries ei ON ei.employer_id = e.id
FULL OUTER JOIN industries i ON i.id = ei.industry_id

WHERE v.salary_from >= 500000 OR v.salary_to >= 500000
ORDER BY i.name
        '''
        
df_7_3 = pd.read_sql_query(query_7_3, connection)
df_7_3

,Название вакансии,Ключевые навыки,Тип рабочего графика,Требования к опыту,Тип трудоустройства,Нижняя граница зарплаты,Верхняя граница зарплаты,Название города,Название работодателя,Название сферы деятельности
0,Брокер по продаже апартаментов,Ведение переговоров\tТелефонные переговоры\tРа...,Полный день,От 1 года до 3 лет,Полная занятость,500000.0,1000000.0,Сочи,Агентство недвижимости ЛЕТО,Агентские услуги в недвижимости
1,Брокер по работе с ключевыми клиентами,Ведение переговоров\tРабота в команде\tГрамотн...,Полный день,Нет опыта,Полная занятость,200000.0,500000.0,Сочи,Агентство недвижимости ЛЕТО,Агентские услуги в недвижимости
2,Директор научно-исследовательского центра (химия),Английский язык\tХимические методы анализа\tЛа...,Полный день,Более 6 лет,Полная занятость,500000.0,NaN,Красноярск,"Август, Фирма","Агрохимия (продвижение, оптовая торговля)"
3,Директор научно-исследовательского центра (химия),Английский язык\tХимические методы анализа\tЛа...,Полный день,Более 6 лет,Полная занятость,500000.0,NaN,Волжский (Волгоградская область),"Август, Фирма","Агрохимия (продвижение, оптовая торговля)"
4,Директор научно-исследовательского центра (химия),Английский язык\tХимические методы анализа\tЛа...,Полный день,Более 6 лет,Полная занятость,500000.0,NaN,Ярославль,"Август, Фирма","Агрохимия (продвижение, оптовая торговля)"
...,...,...,...,...,...,...,...,...,...,...
101,Backend Developer Fiat payments (Node.js Devel...,Node.js\tJava\tTeamleading\tTeam management\tB...,Полный день,От 3 до 6 лет,Полная занятость,200000.0,500000.0,Краснодар,LATOKEN,"Финансово-кредитное посредничество (биржа, бро..."
102,Product manager LATOKEN,Product Management\tBusiness Development\tTeam...,Удаленная работа,От 3 до 6 лет,Полная занятость,400000.0,800000.0,Пермь,LATOKEN,"Финансово-кредитное посредничество (биржа, бро..."
103,Liquidity Bots Developer,"C#\t.NET Framework\tAMQP Rabbit / Kafka, MSA, ...",Удаленная работа,От 3 до 6 лет,Полная занятость,292242.0,584485.0,Краснодар,LATOKEN,"Финансово-кредитное посредничество (биржа, бро..."
104,Liquidity Bots Developer,"C#\t.NET Framework\tAMQP Rabbit / Kafka, MSA, ...",Удаленная работа,От 3 до 6 лет,Полная занятость,292242.0,584485.0,Екатеринбург,LATOKEN,"Финансово-кредитное посредничество (биржа, бро..."


In [55]:
# сгруппируем данные по требованиям к опыту работы и построим график
grouped_experience = df_7_3.groupby('Требования к опыту', as_index=False)['Название вакансии'].count()

fig = px.bar(
    data_frame=grouped_experience,
    x='Требования к опыту',
    y='Название вакансии',
    height=500,
    width=700,
    color = 'Требования к опыту',
    title='Распределение вакансий по требуемому опыту работы'
)

fig.show()

In [56]:

# сгруппируем данные по сферам деятельности и выведем в порядке убывания популярности сфер деятельности

grouped_profile = df_7_3.groupby('Название сферы деятельности', as_index=False)['Название вакансии'].count().sort_values(by='Название вакансии', ascending=False)
display(grouped_profile)

,Название сферы деятельности,Название вакансии
15,Разработка программного обеспечения,23
1,"Агрохимия (продвижение, оптовая торговля)",10
2,Агрохимия (производство),10
3,"Бытовая химия, парфюмерия, косметика (продвиж...",10
4,"Бытовая химия, парфюмерия, косметика (произво...",10
20,"Финансово-кредитное посредничество (биржа, бро...",9
17,"Системная интеграция, автоматизации технологи...",9
7,Кадровые агентства,6
5,"Интернет-компания (поисковики, платежные систе...",4
0,Агентские услуги в недвижимости,2


In [57]:


# узнаем, сколько компаний указали более 4 сфер деятельности и распределим в порядке убывания количества сфер

query_7_4 = f'''SELECT
    e.name "Название компании",
    COUNT(i.id) "Количество сфер деятельности"
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
LEFT JOIN industries i ON i.id = ei.industry_id
GROUP BY e.name
HAVING COUNT(i.id) >= 4
ORDER BY 2 DESC
            '''
            
df_7_4 = pd.read_sql_query(query_7_4, connection)
df_7_4

,Название компании,Количество сфер деятельности
0,Модуль,16
1,Простые решения,11
2,Энергия,10
3,Прайд,10
4,Эксперт,9
...,...,...
2410,"АВТОДОМ - Санкт-Петербург, автоцентр",4
2411,СтройДвор,4
2412,Сухой,4
2413,Люмэкс,4


In [58]:

# вакансии яндекса на удаленке

query_7_5 = f'''SELECT
    v.schedule "Тип рабочего графика",
    COUNT(v.id) "Количество вакансий"
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс'
GROUP BY v.schedule

UNION ALL

SELECT    
    'total',
    COUNT(v.id)
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс'
    
ORDER BY "Количество вакансий" DESC
    '''
    
df_7_5 = pd.read_sql_query(query_7_5, connection)
df_7_5

,Тип рабочего графика,Количество вакансий
0,total,1933
1,Удаленная работа,1690
2,Полный день,131
3,Сменный график,57
4,Гибкий график,55


In [59]:

# количество вакансий и размер зарплаты для каждого уровня опыта работы в DS

query_7_6 = f'''SELECT
    DISTINCT experience,
    ROUND(AVG(COALESCE(((salary_from + salary_to) / 2), salary_from, salary_to))),
    COUNT(id)
FROM 
    vacancies
WHERE (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name LIKE '%ML%' AND name NOT ILIKE '%html%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE '%машинн%обучен%')
GROUP BY experience
'''
df_7_6 = pd.read_sql_query(query_7_6, connection)
df_7_6

,experience,round,count
0,Более 6 лет,NaN,34
1,Нет опыта,74643.0,28
2,От 1 года до 3 лет,139675.0,223
3,От 3 до 6 лет,243115.0,195


In [60]:
query_7_7 = f'''SELECT
    DISTINCT experience "Опыт работы",
    ROUND(AVG(COALESCE(((salary_from + salary_to) / 2), salary_from, salary_to))) "Средняя зарплата"
FROM 
    vacancies
WHERE salary_from IS NOT NULL
    OR salary_to IS NOT NULL
GROUP BY experience
'''

df_7_7 = pd.read_sql_query(query_7_7, connection)
df_7_7

,Опыт работы,Средняя зарплата
0,Более 6 лет,166256.0
1,Нет опыта,40472.0
2,От 1 года до 3 лет,76541.0
3,От 3 до 6 лет,133211.0


In [61]:
# распределение средней зп в зависимости от опыта

fig1 = px.bar(
    data_frame=df_7_7,
    x='Опыт работы',
    y='Средняя зарплата',
    height=500,
    width=700,
    color = 'Опыт работы',
    title='Распределение вакансий по требуемому опыту работы'
)

fig1.show()

In [62]:
# выведем количество вакансий по каждой сфере деятельности в порядке убывания популярности
query_7_8 = f'''SELECT
    i.name "Сфера деятельности",
    COUNT(DISTINCT v.id) "Количество вакансий"
    
FROM vacancies v
JOIN employers e ON e.id = v.employer_id
JOIN employers_industries ei ON ei.employer_id = e.id
JOIN industries i ON i.id = ei.industry_id
GROUP BY i.name
ORDER BY 2 DESC
        '''
        
df_7_8 = pd.read_sql_query(query_7_8, connection)
df_7_8

,Сфера деятельности,Количество вакансий
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
...,...,...
288,Ветеринарная аптека,5
289,Химчистка,4
290,"Ассоциация в сфере культуры, искусства",2
291,Ритуальные услуги,1


<h4>Общий вывод по всем вакансиям.</h4>

В нашей базе данных содержатся сведения о 49197 вакансиях. Можно увидеть, что такая информация, как тип рабочего графика и требования к опыту указаны для всех вакансий без исключения. При этом нижняя граница зарплаты указывается менее, чем в половине случаев - 21485, а верхняя и того реже - 12690. Если рассматривать размеры указанных зарплат, то в среднем нижняя граница составляет 71064 рублей, а верхняя граница - 110 536 рублей. Максимальные значения верхней и нижней границы зарплаты довольно высоки - в обоих случаях 1 000 000 рублей.

Мне стало интересно, как зависит указанная минимальная зарплата от опыта работы, для этого я вывела данные в виде диаграммы, на которой можно увидеть, что с ростом опыта растут и зарплаты, при этом растет и разброс предлагаемых сумм. Примечательно то, что число потенциальных выбросов по зарплате среди вакансий, требующих более 6 лет опыта, гораздо ниже, чем во всех остальных категориях.

Зависимость средней заработной платы от требуемого опыта работы четко прослеживается:
- Нет опыта	- 40472 рубля.
- От 1 года до 3 лет - 76541 рублей.
- От 3 до 6 лет - 133211 рублей.
- Более 6 лет- 166256 рублей.

Я решила посмотреть самые потенциально высокооплачиваемые вакансии. Так как во многих вакансиях не указана никакая граница зарплаты - ни верхняя, ни нижняя, я отфильтровала данные и взяла только те вакансии, в которых хотя бы одна из двух границ указана и составляет от 500000 рублей.
У меня получилась таблица из 106 вакансий. Посмотрим, какие сферы деятельности здесь преобладают, для этого я сгруппировала данные по сфере деятельности и подсчитала количество вакансий для каждой из них.
- На первом месте оказалась Разработка программного обеспечения - 23 вакансии. Кстати, среди полного объема данных это также самая популярная сфера деятельности(12499 вакансий).
- Далее идут: Агрохимия (продвижение, оптовая торговля) - 10 вакансий, Агрохимия (производство) - 10 вакансий, то есть можно увидеть, что в Агрохимии хорошо платят.
- Дальше расположились продвижение и торговля Бытовой химией(10) и производство Бытовой химии(10), здесь аналогично с предыдущими кандидатами - одна сфера, но разные виды деятельности.

При рассмотрении сочетаний типа рабочего графика и типа трудоустройства мое внимание привлекла вакансия, которая сочетает сменный график с проектной работой - 1 вакансия.
Мне стало интересно, что это за работа, заодно я решила посмотреть вакансии, которые включают в себя вахтовый метод с проектной работой(таких всего 2). Выяснилось, что вакансия, совмещающая в себе сменный график с проектной работой - вакансия продюсера, где не требуется опыта, а зарплата составит от 50000 до 70000 рублей. Вот так я открыла для себя что-то новое.
Две вакансии с вахтовым методом и проектной работой - DevOPS Engineer и Инженер по релейной защите и автоматике AutoCAD. В целом эти вакансии, включая ключевые навыки, выглядят вполне правдоподобно.

Воспользуюсь своей уже имеющейся небольшой табличкой самых высокооплачиваемых вакансий для анализа требуемого опыта работы. Я вывела диаграмму, на которой отчетливо видно, что равное количество вакансий требуют опыт работы "от 3 до 6 лет" и "от 6 лет" (49 и 48 вакансий соответственно из общего количества 106). На 6 вакансий готовы взять без опыта работы, а на 3 требуется иметь опыт от 1 года до 3 лет. Стоит заметить, что среди общего объема данных(49197 вакансия) пропорции требуемого стажа совершенно другие: 
- на первом месте там опыт работы 1 года до 3 лет (26152 вакансии),
- за ними следуют "от 3 до 6 лет" - 14511 вакансий,
- совсем без опыта можно выбрать работу среди 7197 вариантов,
- более 6 лет - в 1337 вакансиях.
Вполне логично, что в вакансиях на бОльшую зарплату требуется больший опыт работы.

В самом начале работы над проектом я задалась вопросом, есть ли в нашей базе работодатели, вакансии которых сейчас отсутствуют. Выяснилось, что да, и таких немало, среди общего количества работодателей(23501) есть вакансии лишь от 14766.

Также выяснилось, что у 8419 работодателей из 23501 не указана сфера деятельности. Но зато есть компании, у которых сфера деятельности указана, и не одна! К примеру, 2415 компаний указали более 4 сфер деятельности. Лидирует по числу указанных сфер деятельности компания "Модуль" (16), на втором месте  "Простые решения"(11), на третьем и четвертом - "Энергия" и "Прайд" (указали по 10 сфер деятельности).

Вакансии по городам распределены неравномерно. В то время, как в одних городах их тысячи, почти в половине из имеющихся в нашей базе городов нет ни одной вакансии. Топ-5 городов по количеству вакансий:
- Москва - 5333,
- Санкт-Петербург - 2851,
- Минск - 2112,
- Новосибирск - 2006,
- Алматы - 1892.

При исследовании распределения вакансий Яндекса по городам, когда выяснилось, что вакансии этой компании размещены для 181 региона, мне стало интересно, почему так происходит.  Я обнаружила, что в 1690 из 1933 вакансий указан тип рабочего графика "Удаленная работа". Пазл сошелся, Яндекс ищет сотрудников на удаленную работу, чтобы платить им региональные зарплаты. Что ж, это распространенная практика.

При рассмотрении размеров зарплат в вакансиях DS мое внимание привлекла «потерявшаяся» категория работников с опытом работы от 6 лет. Я выделила из наших баз данные по количеству вакансий в DS с разным уровнем опыта и размером зарплаты для этого уровня. Оказалось, что вакансии с требуемым опытом работы от 6 лет имеются, и их целых 34, однако зарплатная вилка для этих вакансий не указана. Меня это абсолютно не удивляет, так как я знаю, что в IT-вакансиях довольно часто в свободном доступе отсутствуют данные о предлагаемой зарплате.

In [63]:
connection.close()